In [18]:
import pandas as pd
import numpy as np

In [19]:
file_path = "analysis_data/pe_pb_ratio_rolling_analysis.csv"
df = pd.read_csv(file_path, parse_dates=['Date'])

initial_capital = 100_000
tickers = df['TickerName'].unique()
rolling_windows = df['RollingWindow'].unique()

In [20]:
def calculate_max_drawdown(cum_returns):
    running_max = np.maximum.accumulate(cum_returns)
    drawdown = (cum_returns - running_max) / running_max
    return drawdown.min()

def calculate_sharpe_ratio(returns, risk_free_rate=0, periods_per_year=252):
    excess_returns = returns - risk_free_rate / periods_per_year
    if returns.std() == 0:
        return 0
    return np.sqrt(periods_per_year) * excess_returns.mean() / returns.std()

results = []



I used this code to check what colums are present in the dataframe in order to extract Stockprice

In [21]:
print(df.columns.unique().tolist())


['Date', 'TickerName', 'StockPrice_x', 'NetEPS', 'PE', 'StockPrice_y', 'BookValuePerShare', 'PB', 'Rolling_P10_PE', 'Rolling_P90_PE', 'Rolling_P10_PB', 'Rolling_P90_PB', 'Signal', 'RollingWindow']


1. Since I merged the PE and PB ratio datasets, hence both stockprice_x and stockprice_y will have the data.
2. I later also standardized stock price x and y into Stock price

Backtest strategy
Enter on buy signal and stay invested untill the sell signal
2 Types of returns are calculated, strategy and market. Strategy returns are based on the buy and sell signals which are generated using PE and PB as thresholds. Position gets active when the signal is triggered. On the other hand Market returns are calculated using simple buy and hold, no signals are used.

These returns are then used to calculate performance metrics such as ROI, Sharpe Ratio, MDD, ROI to MDD.

Following is the description of the signals:
1. Buy : 1 ( When PE , PB < P10 )
2. Sell : -1 ( When PE , PB > P90 )
3. Hold : 0 

In [22]:
for window in rolling_windows:
    for ticker in tickers:
        temp = df[(df['TickerName'] == ticker) & (df['RollingWindow'] == window)].copy()
        temp = temp.sort_values('Date').reset_index(drop=True)

        if 'StockPrice_x' in temp.columns:
            temp['StockPrice'] = temp['StockPrice_x']
        elif 'StockPrice_y' in temp.columns:
            temp['StockPrice'] = temp['StockPrice_y']
        else:
            raise KeyError("Wrong stock price column")

        temp.drop(columns=['StockPrice_x', 'StockPrice_y'], inplace=True, errors='ignore')

        temp['Return'] = temp['StockPrice'].pct_change().fillna(0)

        temp['Position'] = 0
        holding = False
        for i in range(1, len(temp)):
            if temp.loc[i-1, 'Signal'] == 1:
                holding = True
            elif temp.loc[i-1, 'Signal'] == -1:
                holding = False
            temp.loc[i, 'Position'] = 1 if holding else 0

    
        temp['Strategy_Return'] = temp['Return'] * temp['Position']
        temp['Cumulative_Market'] = (1 + temp['Return']).cumprod()
        temp['Cumulative_Strategy'] = (1 + temp['Strategy_Return']).cumprod()

        roi = temp['Cumulative_Strategy'].iloc[-1] - 1
        max_dd = calculate_max_drawdown(temp['Cumulative_Strategy'].values)
        sharpe = calculate_sharpe_ratio(temp['Strategy_Return'].values)
        roi_to_mdd = roi / abs(max_dd) if max_dd != 0 else np.nan

        results.append({
            'TickerName': ticker,
            'RollingWindow': window,
            'ROI': roi,
            'MaxDrawdown': max_dd,
            'SharpeRatio': sharpe,
            'ROI_to_MDD': roi_to_mdd
        })

print("Backtest Strategy added")

Backtest Strategy added


In [23]:
results_df = pd.DataFrame(results)
results_df.to_csv("analysis_data/experiment_tracking_pe_pb.csv", index=False)
print("Backtesting complete and experiment tracking results are saved")



Backtesting complete and experiment tracking results are saved


Just to check what results are being saved

In [24]:

print(results_df.head())

  TickerName  RollingWindow       ROI  MaxDrawdown  SharpeRatio  ROI_to_MDD
0       AAPL             20  2.183381    -0.395730     3.690176    5.517352
1      GOOGL             20  2.069048    -0.102054     4.342670   20.274143
2       META             20  3.387564    -0.653914     3.961413    5.180443
3       MSFT             20  0.958013    -0.154510     3.503208    6.200340
4       TSLA             20  0.000000     0.000000     0.000000         NaN
